# Algoritmo Branch and Bound


O algoritmo Branch and Bound (Ramificação e Poda) é uma técnica utilizada para resolver problemas de otimização combinatória, como a Programação Linear Inteira Binária. Ele explora o espaço de soluções de forma sistemática, construindo uma árvore de decisões onde cada nó representa um subconjunto do problema original. Abaixo segue os passos:


1. Branching (Ramificação): O problema atual é dividido em subproblemas ao fixar variáveis inteiras em diferentes valores (normalmente 0 ou 1).

2. Bounding (Poda por limites): Cada subproblema é resolvido como um problema de programação linear relaxado (sem restrições inteiras). Se a solução encontrada for pior que a melhor solução inteira conhecida (limite superior), o ramo é descartado (podado).

3. Fathoming (Eliminação): Um nó pode ser eliminado se:

*  **A solução relaxada for inviável.**
*  **A solução relaxada for inteira (ótima para aquele ramo).**
*  **O limite inferior do ramo for pior que o melhor valor atual.**


Esse processo continua até que todos os ramos tenham sido explorados ou podados, garantindo que a melhor solução inteira possível seja encontrada.









In [ ]:
!pip install mip
from mip import Model, xsum, BINARY, MAXIMIZE




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


In [ ]:
from mip import Model, xsum, BINARY, OptimizationStatus, MAXIMIZE, CBC
from collections import deque
import math

# -------------------------------------------------------------
# Função para ler o arquivo de entrada no formato esperado
def ler_arquivo(caminho):
    with open(caminho, 'r') as f:
        linhas = f.readlines()

    # Definição do número de variáveis e restrições
    num_variaveis, num_restricoes = map(int, linhas[0].split())

    #Os Coeficientes da função recebem essa lista
    coef_objetivo = list(map(int, linhas[1].split()))
    restricoes = []

    # Esse laço for irá percorrer as restrições
    for linha in linhas[2:2+num_restricoes]:
        partes = list(map(int, linha.split())) # Converte a linha em uma lista de inteiros
        coef = partes[:-1] # Todos os elementos, exceto o último, são os coeficientes das variáveis
        limite = partes[-1]# O último número é o limite da restrição
        restricoes.append((coef, limite))# Adição da restrição como uma tupla

    #Retorno do número de variáveis, coeficientes da função objetivo e a lista de restrições
    return num_variaveis, coef_objetivo, restricoes


# Modelo de Otimização com Ramificação


A função criar_modelo é responsável por construir um modelo de otimização com foco na maximização da função objetivo, utilizando variáveis binárias. Nela, são definidos os coeficientes da função objetivo, as restrições principais do problema e as restrições adicionais para permitir a ramificação durante o processo do Branch and Bound.

###  Modelo Geral

**Função Objetivo:**

$$\max :\sum_{i=1} C_{i}X_{i}$$

Em que:

* Ci são os coeficientes da função objetivo
*Xi ∈ {0,1} são variáveis binárias



**S.a:**
$$\sum_{i=1} a {ji}X_{i} \leq b_{j} \  \$$


$$x_{k} = vk$$
Sendo:

*  Xk é uma variável binária do problema original, escolhida para ser "ramificada" naquele momento.
*  Vk é um valor fixo (0 ou 1) atribuído à variável binária durante a ramificação.


Em que:


*  aji são os coeficientes das variáveis nas restrições
*  bj é o limite superior da restrição j












In [ ]:
# Função que monta o modelo com restrições adicionais (ramificação)

def criar_modelo(num_var, coef_obj, restricoes, restricoes_extras=[]):
    modelo = Model(sense=MAXIMIZE, solver_name=CBC)

    # Variáveis relaxadas: Binárias, ou seja, entre 0 e 1
    x = [modelo.add_var(var_type=BINARY) for _ in range(num_var)]
    #lb=0, ub=1, por serem variáveis binárias

    # Função Objetivo
    modelo.objective = xsum(coef_obj[i] * x[i] for i in range(num_var))

    # Restrições principais
    for coef, limite in restricoes:
        modelo += xsum(coef[i] * x[i] for i in range(num_var)) <= limite

    #Restrições de ramificação (fixação de variáveis)
    for idx, valor in restricoes_extras:
        modelo += x[idx] == valor

    modelo.optimize()
    return modelo, x


# Variável Ramificada

A função escolher_variavel_fracionaria recebe uma lista de valores e identifica qual deles possui a parte fracionária mais próxima de 0.5. Essa é a heurística utilizada no algoritmo de Branch and Bound para escolher qual variável ramificar.

In [ ]:
# Verifica se todos os valores da solução são inteiros

def solucao_inteira(valores):
    return all(math.isclose(v, round(v)) for v in valores) #math.isclose é uma função do módulo importado math que serve para avaliar se dois valores de ponto flutuantes estão próximos


# Escolhe a variável fracionária mais próxima de 0.5 para ramificar
def escolher_variavel_fracionaria(valores):
    fracionarias = [(i, abs(val - 0.5)) for i, val in enumerate(valores) if not math.isclose(val, round(val))] #lista de tuplas, onde cada tupla contém o índice e a distância até 0.5 de cada valor fracionário presente na lista
    if not fracionarias:
        return None
    return min(fracionarias, key=lambda x: x[1])[0]
    # # A função 'min' com a chave 'lambda x: x[1]' retorna o elemento da lista 'fracionarias' para o qual o segundo elemento da tupla (a distância) é mínimo.
    # [0] no final acessa o primeiro elemento dessa tupla, que é o índice da variável.


# Estratégia de Ramificação

Neste código foi utilizada a busca em profundidade (DFS) por meio da Estrutura de Dados funcionando como uma Pilha. A escolha do DFS se deve à sua característica de explorar um caminho na árvore de busca até a sua profundidade máxima antes de retroceder.

Isso pode levar à descoberta rápida de soluções inteiras viáveis, permitindo que o algoritmo estabeleça limites (bounds) para a otimização e realize a poda.

A poda consiste em descartar ramos da árvore de busca que comprovadamente não podem conter a solução ótima inteira. No código, a poda ocorre quando o valor da função objetivo da solução do problema relaxado (valor_obj) em um determinado nó é pior ou igual ao melhor valor inteiro já encontrado (melhor_valor).



In [ ]:
# Algoritmo principal de Branch-and-Bound

def branch_and_bound(num_var, coef_obj, restricoes):
    melhor_valor = -math.inf ## Melhor valor objetivo encontrado
    melhor_solucao = None

    # Estrutura para busca em profundidade DFS
    abertos = deque()   # Uso da Estrutura de Dados
    abertos.append([])  # Inicia sem restrições extras

    while abertos:
        restricoes_extras = abertos.pop() #pega próximo ramo da pilha
        modelo, vars_ = criar_modelo(num_var, coef_obj, restricoes, restricoes_extras)

        if modelo.status != OptimizationStatus.OPTIMAL:
            continue # Pula se o problema não é solucionável

        valores = [v.x for v in vars_] # Solução atual
        valor_obj = modelo.objective_value  # Valor da função objetivo

        # Poda: Descarte caso seja pior que a melhor solução atual
        if valor_obj <= melhor_valor:
            continue

        #Atualiza a melhor solução
        if solucao_inteira(valores):
            if valor_obj > melhor_valor:
                melhor_valor = valor_obj
                melhor_solucao = [round(v) for v in valores]
        else:
            # Escolha da variável fracionária para ramificar
            j = escolher_variavel_fracionaria(valores) #Chamada da função
            if j is not None:
                abertos.append(restricoes_extras + [(j, 0)])
                abertos.append(restricoes_extras + [(j, 1)])

    return melhor_valor, melhor_solucao

# Conclusão

A combinação estratégica do DFS para a exploração da árvore e da poda para o descarte de ramos ineficientes é fundamental para a performance do algoritmo de Branch-and-Bound.

In [ ]:
# Executa o processo completo para um arquivo de entrada

def executar(caminho_arquivo):
    print(f"\n===> Processando arquivo: {caminho_arquivo}")
    num_var, coef_obj, restricoes = ler_arquivo(caminho_arquivo)
    valor, solucao = branch_and_bound(num_var, coef_obj, restricoes)
    print("Valor ótimo encontrado:", valor)
    print("Solução ótima (binária):", solucao)


    for i, val in enumerate(solucao):
      print(f"x{i+1} = {val}")
print("-" * 30)  # separador entre os testes



# Ponto de entrada do programa

if __name__ == "__main__":
    # Altere os caminhos conforme os arquivos de teste disponíveis
    executar("teste1.txt")
    executar("teste2.txt")
    executar("teste3.txt")
    executar("teste4.txt")

------------------------------

===> Processando arquivo: teste1.txt
Valor ótimo encontrado: 20.0
Solução ótima (binária): [0, 0, 0, 0, 1, 1, 0]
x1 = 0
x2 = 0
x3 = 0
x4 = 0
x5 = 1
x6 = 1
x7 = 0

===> Processando arquivo: teste2.txt
Valor ótimo encontrado: 24.0
Solução ótima (binária): [0, 1, 0, 0, 0, 1, 1, 0, 0]
x1 = 0
x2 = 1
x3 = 0
x4 = 0
x5 = 0
x6 = 1
x7 = 1
x8 = 0
x9 = 0

===> Processando arquivo: teste3.txt
Valor ótimo encontrado: 19.0
Solução ótima (binária): [0, 0, 1, 0, 0, 0, 1, 0, 0]
x1 = 0
x2 = 0
x3 = 1
x4 = 0
x5 = 0
x6 = 0
x7 = 1
x8 = 0
x9 = 0

===> Processando arquivo: teste4.txt
Valor ótimo encontrado: 10.0
Solução ótima (binária): [0, 0, 1, 0, 0, 0, 0, 0, 0]
x1 = 0
x2 = 0
x3 = 1
x4 = 0
x5 = 0
x6 = 0
x7 = 0
x8 = 0
x9 = 0
